# LightGBM Regressor training
- This is an auto-generated notebook.
- To reproduce these results, attach this notebook to a cluster with runtime version **16.2.x-cpu-ml-scala2.12**, and rerun it.
- Compare trials in the [MLflow experiment](#mlflow/experiments/1286397497605697).
- Clone this notebook into your project folder by selecting **File > Clone** in the notebook toolbar.

In [0]:
import mlflow
import databricks.automl_runtime

target_col = "feature"
time_col = "time"

## Load Data

In [0]:
import mlflow
import os
import uuid
import shutil
import pandas as pd

# Create temp directory to download input data from MLflow
input_temp_dir = os.path.join(os.environ["SPARK_LOCAL_DIRS"], "tmp", str(uuid.uuid4())[:8])
os.makedirs(input_temp_dir)


# Download the artifact and read it into a pandas DataFrame
input_data_path = mlflow.artifacts.download_artifacts(run_id="19bc4094750946d7b40fd82895cdb81d", artifact_path="data", dst_path=input_temp_dir)

df_loaded = pd.read_parquet(os.path.join(input_data_path, "training_data"))
# Delete the temp data
shutil.rmtree(input_temp_dir)

# Preview data
display(df_loaded.head(5))

/databricks/python/lib/python3.12/site-packages/databricks/sdk/service/jobs.py:60: SyntaxWarning: invalid escape sequence '\.'
  """The sequence number of this run attempt for a triggered job run. The initial attempt of a run
/databricks/python/lib/python3.12/site-packages/databricks/sdk/service/jobs.py:2570: SyntaxWarning: invalid escape sequence '\.'
  """The sequence number of this run attempt for a triggered job run. The initial attempt of a run
/databricks/python/lib/python3.12/site-packages/databricks/sdk/service/jobs.py:3431: SyntaxWarning: invalid escape sequence '\.'
  """The sequence number of this run attempt for a triggered job run. The initial attempt of a run


time,alimentacao_bu,alimentacao_bs,ceee,%_clinquer,%_gesso_mineral,%__gesso_sintetico,%_calcario,%_po_filler,%_escoria,altura_da_camada_na_mesa,pressao_de_moagem,velocidade_da_mesa,aditivo,agua,dp_moinho,velocidade_sr1,velocidade_fan,vazao_de_gases,relacao_ar_/_po,motores_moinho,exaustor,separador,gases_do_forno,total_vrm,aux,corrente_motor,rotacao 5wa-fn1,torque,potencia_da_mesa,rejeito_da_mesa,horas,producao,blaine,#400,umidade,md1,md2,md3,md4,md5,md6,mda,56a-fn1,5wa-fn1,56a-sr1,energia_-_aditivos_e_corretivos,motores_440,compressores,"transporte,_iluminacao,_etc",energia_total,alimentacao,camada_-_r1,camada_-_r2,camada_-_r3,camada_-_r4,camada_-_r5,camada_-_r6,pressao_-_r1,pressao_-_r2,pressao_-_r3,pressao_-_r4,pressao_-_r5,pressao_-_r6,feature,_automl_split_col_0000
2023-08-01T03:00:00Z,383.8787069,376.0451121,47.04488814,61.0667,3.224712,0.0,4.026222,4.084743,27.59763,25.04673,193.3909,18.96785,142.0,8.14,4.888447,105.8826,85.9,1397.397,222.9621,31.1452,9.371216238,3.57138,0.165,21.83687,10887.75,2000.0,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,1.087224,train
2023-08-01T03:01:00Z,383.8787203,376.045121,47.04488642,61.06669727,3.224711915,0.0,4.026221913,4.084743161,27.59762574,25.04673195,193.3908539,18.96784592,142.0,8.135408401,4.888446808,105.8825531,85.90000153,1397.397095,222.9621304,31.14519838,9.371216015,3.571380033,0.165,21.83686829,10887.75488,2000.0,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,1.087224364,train
2023-08-01T03:02:00Z,383.8787069,376.0451121,47.15391761,61.0667,3.224712,0.0,4.026222,4.084743,27.59763,25.04673,193.3909,18.96785,142.0,8.14,4.888447,105.8826,85.9,1397.397,222.9621,31.1452,9.371216238,3.57138,0.165,21.83687,10887.75,2000.0,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,1.087224,train
2023-08-01T03:03:00Z,383.3345404,375.4931636,45.9289318,60.96062,3.218622,0.0,4.044761,4.110132,27.66586,25.56987,190.2041,18.97711,142.2935,8.13,4.847767,105.9878,86.0,1429.039,228.346,29.91533,9.384991266,3.531356,0.165,27.75906,10682.45,1920.0,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,1.035019,train
2023-08-01T03:04:00Z,383.3345145,375.493134,45.92893571,60.96062268,3.218622595,0.0,4.044761606,4.110132029,27.66586109,25.56986618,190.2041016,18.97710609,142.2935486,8.13,4.847767353,105.9878311,86.0,1429.039185,228.3459771,29.91532733,9.384992003,3.531356058,0.165,27.75906372,10682.44922,1920.767334,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,1.035019398,train


### Select supported columns
Select only the columns that are supported. This allows us to train a model that can predict on a dataset that has extra columns that are not used in training.
`["56a-fn1", "camada_-_r4", "pressao_-_r6", "camada_-_r2", "energia_-_aditivos_e_corretivos", "%__gesso_sintetico", "56a-sr1", "md2", "motores_440", "#400", "camada_-_r6", "pressao_-_r3", "camada_-_r3", "pressao_-_r2", "umidade", "producao", "md3", "pressao_-_r5", "compressores", "alimentacao", "rejeito_da_mesa", "horas", "rotacao
5wa-fn1", "mda", "pressao_-_r4", "pressao_-_r1", "blaine", "5wa-fn1", "potencia_da_mesa", "camada_-_r1", "camada_-_r5", "torque", "md5", "md6", "transporte,_iluminacao,_etc", "energia_total", "md1", "md4"]` are dropped in the pipelines. See the Alerts tab of the AutoML Experiment page for details on why these columns are dropped.

In [0]:
from databricks.automl_runtime.sklearn.column_selector import ColumnSelector
supported_cols = ["exaustor", "%_calcario", "separador", "pressao_de_moagem", "%_escoria", "relacao_ar_/_po", "total_vrm", "ceee", "gases_do_forno", "time", "%_clinquer", "alimentacao_bs", "corrente_motor", "altura_da_camada_na_mesa", "velocidade_fan", "agua", "dp_moinho", "aux", "alimentacao_bu", "velocidade_sr1", "%_po_filler", "aditivo", "motores_moinho", "%_gesso_mineral", "velocidade_da_mesa", "vazao_de_gases"]
col_selector = ColumnSelector(supported_cols)

## Preprocessors

### Datetime Preprocessor
For each datetime column, extract relevant information from the date:
- Unix timestamp
- whether the date is a weekend
- whether the date is a holiday

Additionally, extract extra information from columns with timestamps:
- hour of the day (one-hot encoded)

For cyclic features, plot the values along a unit circle to encode temporal proximity:
- hour of the day
- hours since the beginning of the week
- hours since the beginning of the month
- hours since the beginning of the year

In [0]:
from pandas import Timestamp
from sklearn.compose import ColumnTransformer
from sklearn.pipeline import Pipeline

from databricks.automl_runtime.sklearn import DatetimeImputer
from databricks.automl_runtime.sklearn import OneHotEncoder
from databricks.automl_runtime.sklearn import TimestampTransformer
from sklearn.preprocessing import StandardScaler

imputers = {
  "time": DatetimeImputer(),
}

datetime_transformers = []

for col in ["time"]:
    ohe_transformer = ColumnTransformer(
        [("ohe", OneHotEncoder(sparse=False, handle_unknown="indicator"), [TimestampTransformer.HOUR_COLUMN_INDEX])],
        remainder="passthrough")
    timestamp_preprocessor = Pipeline([
        (f"impute_{col}", imputers[col]),
        (f"transform_{col}", TimestampTransformer()),
        (f"onehot_encode_{col}", ohe_transformer),
        (f"standardize_{col}", StandardScaler()),
    ])
    datetime_transformers.append((f"timestamp_{col}", timestamp_preprocessor, [col]))

### Numerical columns

Missing values for numerical columns are imputed with mean by default.

In [0]:
from sklearn.compose import ColumnTransformer
from sklearn.impute import SimpleImputer
from sklearn.pipeline import Pipeline
from sklearn.preprocessing import FunctionTransformer, StandardScaler

num_imputers = []
num_imputers.append(("impute_mean", SimpleImputer(), ["%_calcario", "%_clinquer", "%_escoria", "%_gesso_mineral", "%_po_filler", "aditivo", "agua", "alimentacao_bs", "alimentacao_bu", "altura_da_camada_na_mesa", "aux", "ceee", "corrente_motor", "dp_moinho", "exaustor", "gases_do_forno", "motores_moinho", "pressao_de_moagem", "relacao_ar_/_po", "separador", "total_vrm", "vazao_de_gases", "velocidade_da_mesa", "velocidade_fan", "velocidade_sr1"]))

numerical_pipeline = Pipeline(steps=[
    ("converter", FunctionTransformer(lambda df: df.apply(pd.to_numeric, errors='coerce'))),
    ("imputers", ColumnTransformer(num_imputers)),
    ("standardizer", StandardScaler()),
])

numerical_transformers = [("numerical", numerical_pipeline, ["exaustor", "%_calcario", "separador", "pressao_de_moagem", "%_escoria", "relacao_ar_/_po", "total_vrm", "ceee", "gases_do_forno", "%_clinquer", "alimentacao_bs", "corrente_motor", "altura_da_camada_na_mesa", "velocidade_fan", "agua", "dp_moinho", "aux", "alimentacao_bu", "velocidade_sr1", "%_po_filler", "aditivo", "motores_moinho", "%_gesso_mineral", "velocidade_da_mesa", "vazao_de_gases"])]

In [0]:
from sklearn.compose import ColumnTransformer

transformers = datetime_transformers + numerical_transformers

preprocessor = ColumnTransformer(transformers, remainder="passthrough", sparse_threshold=0)

## Train - Validation - Test Split
The input data is split by AutoML into 3 sets:
- Train (60% of the dataset used to train the model)
- Validation (20% of the dataset used to tune the hyperparameters of the model)
- Test (20% of the dataset used to report the true performance of the model on an unseen dataset)

`_automl_split_col_0000` contains the information of which set a given row belongs to.
We use this column to split the dataset into the above 3 sets. 
The column should not be used for training so it is dropped after split is done.

Given that `time` is provided as the `time_col`, the data is split based on time order,
where the most recent data is split to the test data.

In [0]:
# AutoML completed train - validation - test split internally and used _automl_split_col_0000 to specify the set
split_train_df = df_loaded.loc[df_loaded._automl_split_col_0000 == "train"]
split_val_df = df_loaded.loc[df_loaded._automl_split_col_0000 == "validate"]
split_test_df = df_loaded.loc[df_loaded._automl_split_col_0000 == "test"]

# Separate target column from features and drop _automl_split_col_0000
X_train = split_train_df.drop([target_col, "_automl_split_col_0000"], axis=1)
y_train = split_train_df[target_col]

X_val = split_val_df.drop([target_col, "_automl_split_col_0000"], axis=1)
y_val = split_val_df[target_col]

X_test = split_test_df.drop([target_col, "_automl_split_col_0000"], axis=1)
y_test = split_test_df[target_col]

## Train regression model
- Log relevant metrics to MLflow to track runs
- All the runs are logged under [this MLflow experiment](#mlflow/experiments/1286397497605697)
- Change the model parameters and re-run the training cell to log a different trial to the MLflow experiment
- To view the full list of tunable hyperparameters, check the output of the cell below

In [0]:
import lightgbm
from lightgbm import LGBMRegressor

help(LGBMRegressor)

Help on class LGBMRegressor in module lightgbm.sklearn:

class LGBMRegressor(sklearn.base.RegressorMixin, LGBMModel)
 |  LGBMRegressor(boosting_type: str = 'gbdt', num_leaves: int = 31, max_depth: int = -1, learning_rate: float = 0.1, n_estimators: int = 100, subsample_for_bin: int = 200000, objective: Union[str, Callable[[Optional[numpy.ndarray], numpy.ndarray], Tuple[numpy.ndarray, numpy.ndarray]], Callable[[Optional[numpy.ndarray], numpy.ndarray, Optional[numpy.ndarray]], Tuple[numpy.ndarray, numpy.ndarray]], Callable[[Optional[numpy.ndarray], numpy.ndarray, Optional[numpy.ndarray], Optional[numpy.ndarray]], Tuple[numpy.ndarray, numpy.ndarray]], NoneType] = None, class_weight: Union[Dict, str, NoneType] = None, min_split_gain: float = 0.0, min_child_weight: float = 0.001, min_child_samples: int = 20, subsample: float = 1.0, subsample_freq: int = 0, colsample_bytree: float = 1.0, reg_alpha: float = 0.0, reg_lambda: float = 0.0, random_state: Union[int, numpy.random.mtrand.RandomState

### Define the objective function
The objective function used to find optimal hyperparameters. By default, this notebook only runs
this function once (`max_evals=1` in the `hyperopt.fmin` invocation) with fixed hyperparameters, but
hyperparameters can be tuned by modifying `space`, defined below. `hyperopt.fmin` will then use this
function's return value to search the space to minimize the loss.

In [0]:
import mlflow
from mlflow.models import Model, infer_signature, ModelSignature
from mlflow.pyfunc import PyFuncModel
from mlflow import pyfunc
import sklearn
from sklearn import set_config
from sklearn.pipeline import Pipeline
from hyperopt import hp, tpe, fmin, STATUS_OK, Trials


# Create a separate pipeline to transform the validation dataset. This is used for early stopping.
pipeline_val = Pipeline([
    ("column_selector", col_selector),
    ("preprocessor", preprocessor),
])

mlflow.sklearn.autolog(disable=True)
pipeline_val.fit(X_train, y_train)
X_val_processed = pipeline_val.transform(X_val)

def objective(params):
  with mlflow.start_run(experiment_id="1286397497605697") as mlflow_run:
    lgbmr_regressor = LGBMRegressor(**params)

    model = Pipeline([
        ("column_selector", col_selector),
        ("preprocessor", preprocessor),
        ("regressor", lgbmr_regressor),
    ])

    # Enable automatic logging of input samples, metrics, parameters, and models
    mlflow.sklearn.autolog(
        log_input_examples=True,
        silent=True,
    )

    model.fit(X_train, y_train, regressor__callbacks=[lightgbm.early_stopping(5), lightgbm.log_evaluation(0)], regressor__eval_set=[(X_val_processed,y_val)])

    
    # Log metrics for the training set
    mlflow_model = Model()
    pyfunc.add_to_model(mlflow_model, loader_module="mlflow.sklearn")
    pyfunc_model = PyFuncModel(model_meta=mlflow_model, model_impl=model)
    training_eval_result = mlflow.evaluate(
        model=pyfunc_model,
        data=X_train.assign(**{str(target_col):y_train}),
        targets=target_col,
        model_type="regressor",
        evaluator_config = {"log_model_explainability": False,
                            "metric_prefix": "training_"  }
    )
    # Log metrics for the validation set
    val_eval_result = mlflow.evaluate(
        model=pyfunc_model,
        data=X_val.assign(**{str(target_col):y_val}),
        targets=target_col,
        model_type="regressor",
        evaluator_config= {"log_model_explainability": False,
                           "metric_prefix": "val_"  }
   )
    lgbmr_val_metrics = val_eval_result.metrics
    # Log metrics for the test set
    test_eval_result = mlflow.evaluate(
        model=pyfunc_model,
        data=X_test.assign(**{str(target_col):y_test}),
        targets=target_col,
        model_type="regressor",
        evaluator_config= {"log_model_explainability": False,
                           "metric_prefix": "test_"  }
   )
    lgbmr_test_metrics = test_eval_result.metrics

    loss = -lgbmr_val_metrics["val_r2_score"]

    # Truncate metric key names so they can be displayed together
    lgbmr_val_metrics = {k.replace("val_", ""): v for k, v in lgbmr_val_metrics.items()}
    lgbmr_test_metrics = {k.replace("test_", ""): v for k, v in lgbmr_test_metrics.items()}

    return {
      "loss": loss,
      "status": STATUS_OK,
      "val_metrics": lgbmr_val_metrics,
      "test_metrics": lgbmr_test_metrics,
      "model": model,
      "run": mlflow_run,
    }

### Configure the hyperparameter search space
Configure the search space of parameters. Parameters below are all constant expressions but can be
modified to widen the search space. For example, when training a decision tree regressor, to allow
the maximum tree depth to be either 2 or 3, set the key of 'max_depth' to
`hp.choice('max_depth', [2, 3])`. Be sure to also increase `max_evals` in the `fmin` call below.

See https://docs.databricks.com/applications/machine-learning/automl-hyperparam-tuning/index.html
for more information on hyperparameter tuning as well as
http://hyperopt.github.io/hyperopt/getting-started/search_spaces/ for documentation on supported
search expressions.

For documentation on parameters used by the model in use, please see:
https://lightgbm.readthedocs.io/en/stable/pythonapi/lightgbm.LGBMRegressor.html

NOTE: The above URL points to a stable version of the documentation corresponding to the last
released version of the package. The documentation may differ slightly for the package version
used by this notebook.

In [0]:
space = {
  "colsample_bytree": 0.6413570877368466,
  "lambda_l1": 1.281286730594843,
  "lambda_l2": 562.0645632662288,
  "learning_rate": 0.45659599940182366,
  "max_bin": 240,
  "max_depth": 2,
  "min_child_samples": 126,
  "n_estimators": 834,
  "num_leaves": 17,
  "subsample": 0.6674565178654761,
  "random_state": 372924344,
}

### Run trials
When widening the search space and training multiple models, switch to `SparkTrials` to parallelize
training on Spark:
```
from hyperopt import SparkTrials
trials = SparkTrials()
```

NOTE: While `Trials` starts an MLFlow run for each set of hyperparameters, `SparkTrials` only starts
one top-level run; it will start a subrun for each set of hyperparameters.

See http://hyperopt.github.io/hyperopt/scaleout/spark/ for more info.

In [0]:
trials = Trials()
fmin(objective,
     space=space,
     algo=tpe.suggest,
     max_evals=1,  # Increase this when widening the hyperparameter search space.
     trials=trials)

best_result = trials.best_trial["result"]
model = best_result["model"]
mlflow_run = best_result["run"]

display(
  pd.DataFrame(
    [best_result["val_metrics"], best_result["test_metrics"]],
    index=pd.Index(["validation", "test"], name="split")).reset_index())

set_config(display="diagram")
model

[LightGBM] [Warning] lambda_l2 is set=562.0645632662288, reg_lambda=0.0 will be ignored. Current value: lambda_l2=562.0645632662288
[LightGBM] [Warning] lambda_l1 is set=1.281286730594843, reg_alpha=0.0 will be ignored. Current value: lambda_l1=1.281286730594843
[LightGBM] [Warning] lambda_l2 is set=562.0645632662288, reg_lambda=0.0 will be ignored. Current value: lambda_l2=562.0645632662288
[LightGBM] [Warning] lambda_l1 is set=1.281286730594843, reg_alpha=0.0 will be ignored. Current value: lambda_l1=1.281286730594843
[LightGBM] [Info] Auto-choosing row-wise multi-threading, the overhead of testing was 0.000741 seconds.
You can set `force_row_wise=true` to remove the overhead.
And if memory is not enough, you can set `force_col_wise=true`.
[LightGBM] [Info] Total Bins 7031
[LightGBM] [Info] Number of data points in the train set: 9423, number of used features: 36
[LightGBM] [Warning] lambda_l2 is set=562.0645632662288, reg_lambda=0.0 will be ignored. Current value: lambda_l2=562.0645

Uploading artifacts:   0%|          | 0/11 [00:00<?, ?it/s]

[LightGBM] [Warning] lambda_l2 is set=562.0645632662288, reg_lambda=0.0 will be ignored. Current value: lambda_l2=562.0645632662288
[LightGBM] [Warning] lambda_l1 is set=1.281286730594843, reg_alpha=0.0 will be ignored. Current value: lambda_l1=1.281286730594843
  0%|          | 0/1 [00:17<?, ?trial/s, best loss=?]

2025/02/27 01:40:53 INFO mlflow.models.evaluation.default_evaluator: Computing model predictions.

2025/02/27 01:40:53 WARNING mlflow.utils.autologging_utils: You are using an unsupported version of langchain. If you encounter errors during autologging, try upgrading / downgrading langchain to a supported version, or try upgrading MLflow.



[LightGBM] [Warning] lambda_l2 is set=562.0645632662288, reg_lambda=0.0 will be ignored. Current value: lambda_l2=562.0645632662288
[LightGBM] [Warning] lambda_l1 is set=1.281286730594843, reg_alpha=0.0 will be ignored. Current value: lambda_l1=1.281286730594843
  0%|          | 0/1 [00:19<?, ?trial/s, best loss=?]

...

[LightGBM] [Warning] lambda_l2 is set=562.0645632662288, reg_lambda=0.0 will be ignored. Current value: lambda_l2=562.0645632662288
[LightGBM] [Warning] lambda_l1 is set=1.281286730594843, reg_alpha=0.0 will be ignored. Current value: lambda_l1=1.281286730594843
  0%|          | 0/1 [00:21<?, ?trial/s, best loss=?]

/databricks/python/lib/python3.12/site-packages/sklearn/metrics/_regression.py:483: FutureWarning: 'squared' is deprecated in version 1.4 and will be removed in 1.6. To calculate the root mean squared error, use the function'root_mean_squared_error'.
  warnings.warn(

2025/02/27 01:40:56 INFO mlflow.models.evaluation.default_evaluator: Testing metrics on first row...

2025/02/27 01:40:56 INFO mlflow.tracking._tracking_service.client: 🏃 View run rebellious-foal-905 at: adb-6229616931294414.14.azuredatabricks.net/ml/experiments/1286397497605697/runs/bde1fd8c306449f282e030378381458a.

2025/02/27 01:40:56 INFO mlflow.tracking._tracking_service.client: 🧪 View experiment at: adb-6229616931294414.14.azuredatabricks.net/ml/experiments/1286397497605697.



100%|██████████| 1/1 [00:21<00:00, 21.93s/trial, best loss: -0.635514969395893]


split,score,example_count,mean_absolute_error,mean_squared_error,root_mean_squared_error,sum_on_target,mean_on_target,r2_score,max_error,mean_absolute_percentage_error
validation,0.635514969395893,3141,0.08952204019071838,0.014082435658540435,0.11866943860379738,3231.0786052189983,1.028678320668258,0.635514969395893,1.456790592798919,3.108567464240857E12
test,0.5452066265679875,3141,0.08550862761826482,0.01366552029413502,0.11689961631303594,3470.7827868030045,1.1049929279856747,0.5452066265679875,1.6887229301413402,1.1871242660659634E12


Pipeline(steps=[('column_selector',
                 ColumnSelector(cols=['exaustor', '%_calcario', 'separador',
                                      'pressao_de_moagem', '%_escoria',
                                      'relacao_ar_/_po', 'total_vrm', 'ceee',
                                      'gases_do_forno', 'time', '%_clinquer',
                                      'alimentacao_bs', 'corrente_motor',
                                      'altura_da_camada_na_mesa',
                                      'velocidade_fan', 'agua', 'dp_moinho',
                                      'aux', 'alimentacao_bu', 'velocidade_sr1',
                                      '%_po_filler', '...
                                                   'motores_moinho',
                                                   '%_gesso_mineral',
                                                   'velocidade_da_mesa',
                                                   'vazao_de_gases'])])),
                ('regressor',
                 LGBMRegressor(colsample_bytree=0.6413570877368466,
                               lambda_l1=1.281286730594843,
                               lambda_l2=562.0645632662288,
                               learning_rate=0.45659599940182366, max_bin=240,
                               max_depth=2, min_child_samples=126,
                               n_estimators=834, num_leaves=17,
                               random_state=372924344,
                               subsample=0.6674565178654761))])

### Patch pandas version in logged model

Ensures that model serving uses the same version of pandas that was used to train the model.

In [0]:
import mlflow
import os
import shutil
import tempfile
import yaml

run_id = mlflow_run.info.run_id

# Set up a local dir for downloading the artifacts.
tmp_dir = tempfile.mkdtemp()

client = mlflow.tracking.MlflowClient()

# Fix conda.yaml
conda_file_path = mlflow.artifacts.download_artifacts(artifact_uri=f"runs:/{run_id}/model/conda.yaml", dst_path=tmp_dir)
with open(conda_file_path) as f:
  conda_libs = yaml.load(f, Loader=yaml.FullLoader)
pandas_lib_exists = any([lib.startswith("pandas==") for lib in conda_libs["dependencies"][-1]["pip"]])
if not pandas_lib_exists:
  print("Adding pandas dependency to conda.yaml")
  conda_libs["dependencies"][-1]["pip"].append(f"pandas=={pd.__version__}")

  with open(f"{tmp_dir}/conda.yaml", "w") as f:
    f.write(yaml.dump(conda_libs))
  client.log_artifact(run_id=run_id, local_path=conda_file_path, artifact_path="model")

# Fix requirements.txt
venv_file_path = mlflow.artifacts.download_artifacts(artifact_uri=f"runs:/{run_id}/model/requirements.txt", dst_path=tmp_dir)
with open(venv_file_path) as f:
  venv_libs = f.readlines()
venv_libs = [lib.strip() for lib in venv_libs]
pandas_lib_exists = any([lib.startswith("pandas==") for lib in venv_libs])
if not pandas_lib_exists:
  print("Adding pandas dependency to requirements.txt")
  venv_libs.append(f"pandas=={pd.__version__}")

  with open(f"{tmp_dir}/requirements.txt", "w") as f:
    f.write("\n".join(venv_libs))
  client.log_artifact(run_id=run_id, local_path=venv_file_path, artifact_path="model")

shutil.rmtree(tmp_dir)

Adding pandas dependency to conda.yaml


Adding pandas dependency to requirements.txt


## Feature importance

SHAP is a game-theoretic approach to explain machine learning models, providing a summary plot
of the relationship between features and model output. Features are ranked in descending order of
importance, and impact/color describe the correlation between the feature and the target variable.
- Generating SHAP feature importance is a very memory intensive operation, so to ensure that AutoML can run trials without
  running out of memory, we disable SHAP by default.<br />
  You can set the flag defined below to `shap_enabled = True` and re-run this notebook to see the SHAP plots.
- To reduce the computational overhead of each trial, a single example is sampled from the validation set to explain.<br />
  For more thorough results, increase the sample size of explanations, or provide your own examples to explain.
- SHAP cannot explain models using data with nulls; if your dataset has any, both the background data and
  examples to explain will be imputed using the mode (most frequent values). This affects the computed
  SHAP values, as the imputed samples may not match the actual data distribution.

For more information on how to read Shapley values, see the [SHAP documentation](https://shap.readthedocs.io/en/latest/example_notebooks/overviews/An%20introduction%20to%20explainable%20AI%20with%20Shapley%20values.html).

> **NOTE:** SHAP run may take a long time with the datetime columns in the dataset.

In [0]:
# Set this flag to True and re-run the notebook to see the SHAP plots
shap_enabled = True

In [0]:
if shap_enabled:
    mlflow.autolog(disable=True)
    mlflow.sklearn.autolog(disable=True)
    from shap import KernelExplainer, summary_plot
    # SHAP cannot explain models using data with nulls.
    # To enable SHAP to succeed, both the background data and examples to explain are imputed with the mode (most frequent values).
    mode = X_train.mode().iloc[0]

    # Sample background data for SHAP Explainer. Increase the sample size to reduce variance.
    train_sample = X_train.sample(n=min(100, X_train.shape[0]), random_state=372924344).fillna(mode)

    # Sample some rows from the validation set to explain. Increase the sample size for more thorough results.
    example = X_val.sample(n=min(100, X_val.shape[0]), random_state=372924344).fillna(mode)

    # Use Kernel SHAP to explain feature importance on the sampled rows from the validation set.
    predict = lambda x: model.predict(pd.DataFrame(x, columns=X_train.columns))
    explainer = KernelExplainer(predict, train_sample, link="identity")
    shap_values = explainer.shap_values(example, l1_reg=False, nsamples=500)
    summary_plot(shap_values, example)

[LightGBM] [Warning] lambda_l2 is set=562.0645632662288, reg_lambda=0.0 will be ignored. Current value: lambda_l2=562.0645632662288
[LightGBM] [Warning] lambda_l1 is set=1.281286730594843, reg_alpha=0.0 will be ignored. Current value: lambda_l1=1.281286730594843


  0%|          | 0/100 [00:00<?, ?it/s]

[LightGBM] [Warning] lambda_l2 is set=562.0645632662288, reg_lambda=0.0 will be ignored. Current value: lambda_l2=562.0645632662288
[LightGBM] [Warning] lambda_l1 is set=1.281286730594843, reg_alpha=0.0 will be ignored. Current value: lambda_l1=1.281286730594843
[LightGBM] [Warning] lambda_l2 is set=562.0645632662288, reg_lambda=0.0 will be ignored. Current value: lambda_l2=562.0645632662288
[LightGBM] [Warning] lambda_l1 is set=1.281286730594843, reg_alpha=0.0 will be ignored. Current value: lambda_l1=1.281286730594843
[LightGBM] [Warning] lambda_l2 is set=562.0645632662288, reg_lambda=0.0 will be ignored. Current value: lambda_l2=562.0645632662288
[LightGBM] [Warning] lambda_l1 is set=1.281286730594843, reg_alpha=0.0 will be ignored. Current value: lambda_l1=1.281286730594843
[LightGBM] [Warning] lambda_l2 is set=562.0645632662288, reg_lambda=0.0 will be ignored. Current value: lambda_l2=562.0645632662288
[LightGBM] [Warning] lambda_l1 is set=1.281286730594843, reg_alpha=0.0 will be 

## Inference
[The MLflow Model Registry](https://docs.databricks.com/applications/mlflow/model-registry.html) is a collaborative hub where teams can share ML models, work together from experimentation to online testing and production, integrate with approval and governance workflows, and monitor ML deployments and their performance. The snippets below show how to add the model trained in this notebook to the model registry and to retrieve it later for inference.

> **NOTE:** The `model_uri` for the model already trained in this notebook can be found in the cell below

### Register to Model Registry
```
model_name = "Example"

model_uri = f"runs:/{ mlflow_run.info.run_id }/model"
registered_model_version = mlflow.register_model(model_uri, model_name)
```

### Load from Model Registry
```
model_name = "Example"
model_version = registered_model_version.version

model_uri=f"models:/{model_name}/{model_version}"
model = mlflow.pyfunc.load_model(model_uri=model_uri)
model.predict(input_X)
```

### Load model without registering
```
model_uri = f"runs:/{ mlflow_run.info.run_id }/model"

model = mlflow.pyfunc.load_model(model_uri=model_uri)
model.predict(input_X)
```

In [0]:
# model_uri for the generated model
print(f"runs:/{ mlflow_run.info.run_id }/model")

runs:/72706d7da6674b41b0a8bc713b827237/model
